In [1]:
import psycopg2
import glob
import pandas as pd
import os
import subprocess
import warnings
warnings.filterwarnings('ignore')

In [34]:
# Get database credentials as dictionary
result = subprocess.run(['heroku', 'pg:credentials:url', 'DATABASE',
						 '--name', 'default', '-a', 'disch'], stdout=subprocess.PIPE, shell = True)

cred_str = result.stdout.split(b'\n')[2].decode("utf-8").strip().strip('"')
cred_str_list = cred_str.split(" ")
creds = {}
for cred in cred_str_list:
	cred_name, cred_val = cred.split('=')
	creds[cred_name] = cred_val

In [35]:
# accessing the credentials
database_name = creds['dbname']
host = creds['host']
port = creds['port']
user = creds['user']
password = creds['password']

In [36]:
# connect to database
connection = psycopg2.connect(host=host, database=database_name, user=user, password=password, port=port)
cursor = connection.cursor()

In [37]:
# #fetching all database tables names 
# cursor.execute("""SELECT table_name FROM information_schema.tables
# 	   WHERE table_schema = 'public' LIMIT 5""")
# for table in cursor.fetchall():
# 	print("Table name:", table[0])
	
# 	PostgreSQL_select_Query = 'select * from "{}"'.format(table[0])
# 	cursor.execute(PostgreSQL_select_Query)
# 	d = pd.read_sql_query(PostgreSQL_select_Query, connection)
# 	print(pd.DataFrame(d))

In [41]:
# from protoype question to answer 
PostgreSQL_select_Query = '''SELECT "DiscH_prototype_question".description, "DiscH_prototype_answer".answer_category_num
							FROM "DiscH_prototype_question"
							JOIN "DiscH_prototype_answer" 
							ON "DiscH_prototype_question".question_id = "DiscH_prototype_answer".question_id_id '''


# from protoype question to answer bow 
PostgreSQL_select_Query_BOW = '''SELECT * 
							FROM "DiscH_prototype_question"
							JOIN "DiscH_prototype_answer_bow"
							ON "DiscH_prototype_question".question_id = "DiscH_prototype_answer_bow".question_id_id '''


# executing 
cursor.execute(PostgreSQL_select_Query)
cursor.execute(PostgreSQL_select_Query_BOW)


data = pd.read_sql_query(PostgreSQL_select_Query, connection)
data_BOW = pd.read_sql_query(PostgreSQL_select_Query_BOW, connection)

# converting the sql tables to dataframe
dataframe = pd.DataFrame(data)
dataframe_BOW = pd.DataFrame(data_BOW)

In [42]:
# saving the data
dataframe.to_excel('../data/responses_data.xlsx', index=False)
dataframe_BOW.to_excel('../data/BOW_data.xlsx', index=False)

In [11]:
# closing the connection
connection.close()